In [ ]:
!pip install ultralytics

In [ ]:
!pip install supervision

In [12]:
from ultralytics import YOLO
import numpy as np
import matplotlib.pyplot as plt
import cv2
import supervision as sv

In [4]:
source = "/content/drive/MyDrive/capstone_data/capstone2024/virat/video1.mp4"
output = "compressed_video.mp4"

In [5]:
def cut_frames(video:str, compress:float, output_file="") -> None:
  """
  :video: path to the source video.
  :compress: skips x portion of every 10 frames
  output_file: path
  """
  output_file += "cut_video.mp4"
  cap = cv2.VideoCapture(video)
  droprate = (1 - compress) * 10
  if not cap.isOpened():
      print("Error: Could not open the input video file.")
      exit()


  fourcc = cv2.VideoWriter_fourcc(*'XVID')  # Change this to your desired codec
  frame_size = (int(cap.get(cv2.CAP_PROP_FRAME_WIDTH)), int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT)))
  frame_rate = int(cap.get(cv2.CAP_PROP_FPS))

  out = cv2.VideoWriter(output_file, fourcc, frame_rate, frame_size, isColor=True)

  if not out.isOpened():
      print("Error: Could not create the output video file.")
      cap.release()
      exit()

  counter = 0
  while cap.isOpened():
      ret, frame = cap.read()

      if not ret:
          break

      if counter % droprate != 0:
        out.write(frame)
      counter+=1

  cap.release()
  out.release()

In [6]:
cut_frames(source, 0.3)

In [7]:
output_file = "cut_video.mp4"

In [25]:
model = YOLO("/content/yolov8n.pt")

In [20]:
model.info()

YOLOv8n summary: 225 layers, 3013188 parameters, 0 gradients, 8.2 GFLOPs


(225, 3013188, 0, 8.2059776)

In [ ]:
#video detectiona and visualization
#tracking

tracker = sv.ByteTrack(lost_track_buffer=10, frame_rate=30)
box_annotator = sv.BoundingBoxAnnotator()

def callback(frame: np.ndarray, _: int) -> np.ndarray:
    # set target class
    #, classes=[0]
    results = model(frame)[0]
    detections = sv.Detections.from_ultralytics(results)
    detections = tracker.update_with_detections(detections)
    return box_annotator.annotate(frame.copy(), detections=detections)

sv.process_video(
    source_path=source,
    target_path="result.mp4",
    callback=callback
)

In [27]:
!mv result.mp4 /content/drive/MyDrive/capstone_data

##pseudocode for integration of components


In [ ]:
def get_bbox(detection):
  """
  takes yolo detection result as an input
  returns bounding boxes for objects
  in form of [x, y,
              m, n]
  """
  pass

In [ ]:
def get_object_img(bbox):
  x,y, h, w = 1, 1, 1, 1
  img = cv2.imread("lenna.png")
  crop_img = img[y:y+h, x:x+w]
  return crop_img